In [5]:
import numpy as np
import cmath
from scipy.optimize import curve_fit
import tensorflow as tf
from tensorflow import keras

In [2]:
# 下载数据集
minist = tf.keras.datasets.mnist
(train_x, train_y), (test_x, test_y) = minist.load_data()

In [3]:
# 输出数据相关信息
print("Training set:", len(train_x))
print("Testing set:", len(test_x))

Training set: 60000
Testing set: 10000


In [4]:
print("train_x", train_x.shape, train_x.dtype)
print("train_y", train_y.shape, train_y.dtype)
shape0, shape1, shape2 = train_x.shape

train_x (60000, 28, 28) uint8
train_y (60000,) uint8


In [6]:
new_train_x = np.zeros((shape0, shape1 * shape2))
for i in range(shape0):
    new_train_x[i] = train_x[i].flatten()
print(new_train_x.shape)

(60000, 784)


Given data $(x_1,y_1), \cdots, (x_n,y_n)$ from $\mathbb{R}^d \times \mathbb{R}$, we find the predictor $h_{n,N} \in \mathcal{H}_N$ via ERM with squared loss.

In [25]:
class RFF:
    __d = 0
    __N = 0
    
    def __init__(self, d: int, N: int) -> None:
        self.__d = d
        self.__N = N

    @classmethod
    def fit(self, x: np.ndarray, y: np.ndarray):
        return curve_fit(self.__h, x, y)

    @classmethod
    def __phi(self, x: np.ndarray, v: np.ndarray) -> any:
        '''
        The kernel function

        Input:
        - x: The input data
        - v: The auxiliary matrix

        Returns:
        - The result of the function
        '''

        return np.exp(cmath.sqrt(-1) * np.dot(v, x))
    
    @classmethod
    def __h(self, x: np.ndarray, a: np.ndarray) -> int:
        '''
        Map the input image to its label with Random Fourier Features.

        Inputs:
        - x: The matrix of the image
        - N: The number of features 
        - a: The array of parameters

        Returns:
        - The predicted label
        '''

        v = np.random.randn(self.__N, self.__d)
        result = 0
        for i in range(self.__N):
            result += (a[i] * self.__phi(x, v[i]))
        return result

In [8]:
d = train_x.shape[1] * train_x.shape[2] # 数据维度
maxN = 10 # 设置最大特征数

In [26]:
rff = RFF(d, 10)
result = rff.fit(new_train_x, train_y)

In [6]:
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[shape1, shape2]))
model.add(keras.layers.Dense(300, activation="relu"))
model.add(keras.layers.Dense(100, activation="relu"))
model.add(keras.layers.Dense(10, activation="softmax"))
model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer="sgd",
    metrics=["accuracy"]
)
history = model.fit(train_x, train_y, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 3s 2ms/step - loss: 736076160.0000 - accuracy: 0.1119
Epoch 2/5
1875/1875 [==============================] - 3s 2ms/step - loss: 2.3013 - accuracy: 0.1124
Epoch 3/5
1875/1875 [==============================] - 3s 2ms/step - loss: 2.3013 - accuracy: 0.1124
Epoch 4/5
1875/1875 [==============================] - 3s 2ms/step - loss: 2.3013 - accuracy: 0.1124
Epoch 5/5
1875/1875 [==============================] - 3s 2ms/step - loss: 2.3013 - accuracy: 0.1124


In [7]:
model.evaluate(test_x, test_y)

313/313 [==============================] - 0s 1ms/step - loss: 2.5490 - accuracy: 0.1135


[2.549006462097168, 0.11349999904632568]